# Librerias necesarias para el funcionamiento

In [1]:
!pip install --upgrade gdown
!python -m spacy download es_core_news_lg
import spacy
import es_core_news_lg
from collections import Counter
import re
import csv
import gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.0/568.0 MB 2.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Descargar Archivos

In [2]:
#tóxico archivos csv

!gdown 1gmV34z9n5URFOHBbbWtP-eNLjLWdtXNE #toxico_2011.csv
!gdown 1YdSD8firXalyWeepu0hwuGCYo5EDj47v #toxico_2022.csv

Downloading...
From: https://drive.google.com/uc?id=1gmV34z9n5URFOHBbbWtP-eNLjLWdtXNE
To: /content/toxico_2011.csv
100% 2.79M/2.79M [00:00<00:00, 145MB/s]
Downloading...
From: https://drive.google.com/uc?id=1YdSD8firXalyWeepu0hwuGCYo5EDj47v
To: /content/toxico_2022.csv
100% 2.83M/2.83M [00:00<00:00, 74.1MB/s]


In [3]:
#unicornio archivos csv

!gdown 10YiOpKeUswK74jijk2Go34cLVfN8pNa2 #unicornio_2011.csv
!gdown 1RG_7b4IiKXR_bkH3KQTvzsajCyQmKuYh #unicornio_2022.csv

Downloading...
From: https://drive.google.com/uc?id=10YiOpKeUswK74jijk2Go34cLVfN8pNa2
To: /content/unicornio_2011.csv
100% 2.01M/2.01M [00:00<00:00, 78.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RG_7b4IiKXR_bkH3KQTvzsajCyQmKuYh
To: /content/unicornio_2022.csv
100% 2.86M/2.86M [00:00<00:00, 58.3MB/s]


In [4]:
#hibernar archivos csv

!gdown 1xboi6KBAuQx-Fe8iM0GVQ7i3BlNcfAMR #hibernar_2011.csv
!gdown 1pxrNqFVR4QDcu1VlUFdbnHeGyroDNqa9 #hibernar_2022.csv

Downloading...
From: https://drive.google.com/uc?id=1xboi6KBAuQx-Fe8iM0GVQ7i3BlNcfAMR
To: /content/hibernar_2011.csv
100% 943k/943k [00:00<00:00, 69.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pxrNqFVR4QDcu1VlUFdbnHeGyroDNqa9
To: /content/hibernar_2022.csv
100% 479k/479k [00:00<00:00, 48.1MB/s]


In [5]:
#viral archivos csv

!gdown 15ta-Z7Li1uaIiTbiXQxCdoIa4anaf-9i #viral_2011.csv
!gdown 1slxwNFVEyMHLQ1ZGmj6hGz7Y7CJidTKa #viral_2022.csv

Downloading...
From: https://drive.google.com/uc?id=15ta-Z7Li1uaIiTbiXQxCdoIa4anaf-9i
To: /content/viral_2011.csv
100% 2.80M/2.80M [00:00<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=1slxwNFVEyMHLQ1ZGmj6hGz7Y7CJidTKa
To: /content/viral_2022.csv
100% 2.84M/2.84M [00:00<00:00, 139MB/s]


# Leer los archivos

In [6]:
def abrir_csv(file_path) -> str:
    try:
        with open(file_path, 'r', encoding='utf-8', newline='') as csvfile:
            reader = csv.reader(csvfile)
            # Ignoramos las primeras cinco lineas que solo dan información de sketch engine
            for _ in range(5):
                next(reader)
            # Solo queremos guardar los contextos, no hace falta el origen, etc.
            content = ' '.join([' '.join(row[1:]) for row in reader])

            # Eliminamos elementos de formato no necesarios para el estudio
            symbol_pattern = re.compile(r'[><|]|/s><s')
            content = re.sub(symbol_pattern, '', content)

            return content
    #En caso de error
    except FileNotFoundError:
        return f"El archivo '{file_path}' no existe."
    except Exception as e:
        return f"Ha ocurrido un error: {e}"

In [7]:
#toxico

toxico_11_csv = "toxico_2011.csv"
toxico_22_csv = "toxico_2022.csv"
toxico_11 = abrir_csv(toxico_11_csv)
toxico_22 = abrir_csv(toxico_22_csv)
print(toxico_22[:1000]) #comprobar formato

s A su juicio, la corrupción forma parte de una cultura social.  El corrupto es un depredador de la sociedad, tiene hábitos tóxicos , es deshonesto, no conoce límites ni ve consecuencias de sus actos, analiza.  Esta patología de tipo social tiene daños .  La contaminación del aire ambiente -incluidos los contaminantes potencialmente dañinos como las PM2,5 y los gases tóxicos emitidos por las industrias, los hogares y los vehículos- puede aumentar la inflamación y el estrés oxidativo en el ), obesidad, sedentarismo, tabaquismo, enfermedades del corazón que alteran el ritmo cardíaco así como el consumo de tóxicos y la toma de anticonceptivos orales.  Los especialistas advierten que también se dan ictus en gente joven.  Ocurren por de tomar suplementos.  Ahora bien, como recuerda Piñeiro, "no se puede obviar que la vitamina D en altas dosis es tóxica ".  En opinión de López, esta ''sobreimportancia'' a la vitamina D puede deberse a que "todo lo que rodea al mundo del hueso, para experimen

In [8]:
#unicornio

unicornio_11_csv = "unicornio_2011.csv"
unicornio_22_csv = "unicornio_2022.csv"
unicornio_11 = abrir_csv(unicornio_11_csv)
unicornio_22 = abrir_csv(unicornio_22_csv)
print(unicornio_11[:1000])

, de lento caminar, entre la calima del desierto, hizo pensar a muchos viajeros que se encontraban ante el buscado unicornio .  En el siglo IV antes de Cristo, Aristóteles ya asociaba el oryx de Arabia (Oryx leucoryx) con la mítica criatura. /s su existencia, por lo tanto han pasado a ser considerados personajes de leyenda o ficción, como lo son los unicornios , el Fary o Spock.  ¿Sabías que...?  ...los pelos de polla son muy poco nutritivos ?  Te puedes comer un_millón y no engordas los que luchan por MonEsVol .  Ahí entra cualquiera que luche por Dios, principalmente en contra de la Inciclopedia y su Unicornio Rosa Invisible.  Dante se encuentra en este lugar con su tatarabuelo, quién le dice que lo van a correr de su casa por vestuario de la peli "Las vírgenes suicidas" y en la foto no se ve, pero lo que me ha ganado definitivamente es que tiene un unicornio blanco en la parte de atrás.  Puede llegar a ser tan reconocible como el bikini de la piña de Chloé de hace algunos años. /s c

In [9]:
#hibernar

hibernar_11_csv = "hibernar_2011.csv"
hibernar_22_csv = "hibernar_2022.csv"
hibernar_11 = abrir_csv(hibernar_11_csv)
hibernar_22 = abrir_csv(hibernar_22_csv)
print(hibernar_22[:1000])

Alsa, IAG 7 y Gowall.  El Gobierno confirmó en septiembre la suspensión de los viajes del Imserso.  Llevaban varios meses hibernados por la crisis sanitaria.  Este año, al no haber concluido la campaña de vacunación y al no tener controlado el .  Al ejecutar el archivo aparecerá una ventana con muchas acciones a realizar: apagar, reiniciar, suspender, hibernar , bloquear y cerrar la sesión.  El proceso de configuración es el mismo para todas las opciones así que vamos a probar con la mayor parte de su existencia en Estados Unidos y Canadá, y en época invernal migra a tierras mexicanas; para poder hibernar recorre alrededor de 4_mil_200 kilómetros en esta travesía y descansa en los cálidos bosques de oyamel, pino, encino y en griego significa "transformación somnolienta".  Este nombre hace referencia a la capacidad que tiene la especie de hibernar y transformarse de oruga a una bella mariposa color naranja vibrante.  Las mariposas nacidas durante este periodo de agua.  En lo que respect

In [10]:
#viral

viral_11_csv = "viral_2011.csv"
viral_22_csv = "viral_2022.csv"
viral_11 = abrir_csv(viral_11_csv)
viral_22 = abrir_csv(viral_22_csv)
print(viral_22[:1000])

rottweiler reciben más de tres dosis de la vacuna polivalente (V8 o V10) ya que su predisposición a enfermedades virales es tan grande.  El Rottweiler tiene una reputación de ser valiente, pero lo que pocos saben es que es un excelente perro de .  "Porque es mi vida privada, no pública", lo que deja ver un poco de molestia del cantante.  El comentario se volvió viral inmediatamente y fue replicado por páginas en redes que se encargan de seguirle los pasos a las personalidades de la .  Seguramente con un producto a base de mRna incluso para aquellos que ya han sido inmunizados con una vacuna de vector viral como AstraZeneca o Janssen.  Ulloa: resistencia a cambiar, el gran problema de Panamá  El arzobispo José Domingo Ulloa se posaran en él.  Dibu fue héroe de la Selección con un equilibrio justo entre sus magníficas condiciones, un trash talk viral y algunos métodos alternativos para un futbolista como el yoga, los pilates o la psicología.  A la tradicional que una persona se infecta c

# Palabras más comunes

In [11]:
nlp = es_core_news_lg.load()
nlp.max_length = 3000000

#Esta función devuelve 50 palabras de todo tipo (excepto palabras vacías y puntuación) según su frecuencia junto al neologismo
def get_words(file):
    doc = nlp(file)

    lemmatized_words = [token.lemma_
                        for token in doc
                        if not token.is_stop and not token.is_punct]

    word_freq = Counter(lemmatized_words)
    common_words = word_freq.most_common(50)
    return common_words

#Muy similar a la anterior pero en este caso solo sustantivos
def get_nouns(file, avoid_word):
    doc = nlp(file)

    lemmatized_nouns = [token.lemma_
                        for token in doc
                        if (not token.is_stop and
                            not token.is_punct and
                            token.pos_ == "NOUN" and
                            token.lemma_ != avoid_word)]

    noun_freq = Counter(lemmatized_nouns)
    common_nouns = noun_freq.most_common(50)

    # Enumerate and print the results
    for index, (noun, frequency) in enumerate(common_nouns, start=1):
        print(f"{index}. {noun}: {frequency}")

In [13]:
#Para cambiar de neologismo solo eliminar el término en las casilla
# a la derecha llamada neo
#Opciones: tóxico, unicornio, hibernar


neo = "hibernar" # @param {type:"string"}
if neo == "tóxico":
    corpus = "toxico"
elif neo == "hibernar" or neo == "unicornio":
    corpus = neo
else:
    print("Por favor, introduzca un neologismo válido.")

corpus_11 = str(corpus) + "_11"
corpus_22 = str(corpus) + "_22"
evitar_neologismo = neo

In [ ]:
# corpus 2011
print(f"Buscando los sustantivos más frecuentes en el contexto general de {evitar_neologismo} en 2011...")
sustantivos_11 = get_nouns(globals()[corpus_11], evitar_neologismo)
print(sustantivos_11)

In [ ]:
# corpus 2021-22
print(f"Buscando los sustantivos más frecuentes en el contexto general de {evitar_neologismo} en 2021-22...")
sustantivos_22 = get_nouns(globals()[corpus_22], evitar_neologismo)
print(sustantivos_22)

# Dependencia Sintáctica

In [19]:
#Esta función coge el lemma del neologismo
def get_dep_sus(file, target_w):
    doc = nlp(file)

    target_word = target_w
    target_lemma = nlp(target_word)[0].lemma_

    linked_words = []

    for token in doc:
        if not token.is_stop and not token.is_punct and token.pos_ == "NOUN" and token.head.lemma_ == target_lemma:
            linked_words.append((token.lemma_, token.dep_))

    word_frequency = Counter(linked_words)

    print(f"Top 25 palabras relacionadas con el lemma'{target_word}': \n")
    for (word, dep), count in word_frequency.most_common(25):
        print(f"'{word}' funciona como {dep}: {count} veces")
        # Print example sentences for the word
        #print("Example sentences:")
        #example_sentence_counter = 0
        #for sent in doc.sents:
            #if word in [token.lemma_ for token in sent]:
               # print(sent.text)
                #example_sentence_counter += 1
                #if example_sentence_counter >= 2:  # Change to limit the number of example sentences per word
                   # break  # Stop after printing the specified number of example sentences
        #print()

def get_suj_neo(file, target_w):
    doc = nlp(file)

    target_word = target_w
    target_lemma = nlp(target_word)[0].lemma_

    linked_words = []
    for token in doc:
        if not token.is_stop and not token.is_punct and token.pos_ == "NOUN" and token.head.lemma_ == target_lemma and token.dep_ == "nsubj":
            linked_words.append((token.lemma_, token.dep_))
    word_frequency = Counter(linked_words)

    print(f"Top 25 palabras 'sujeto' en relacion con '{target_word}': \n")
    for (word, dep), count in word_frequency.most_common(25):
        print(f"'{word}' funciona como {dep}: {count} veces")

In [14]:
neologismo = neo

In [ ]:
get_dep_sus(globals()[corpus_11], neologismo)

In [ ]:
#esta funcion sirve para comprobar solamente los sustantivos etiquetados con nsubj

#get_suj_neo(globals()[corpus_11], neologismo)

In [ ]:
get_dep_sus(globals()[corpus_22], neologismo)

In [ ]:
#get_suj_neo(globals()[corpus_22], neologismo)

In [17]:
#se ha usado esta función para buscar contextos donde ocurran una palabra especifica y el neologismo
def checker(file, target_w1, target_w2):
    doc = nlp(file)

    target_lemma1 = nlp(target_w1)[0].lemma_
    target_lemma2 = nlp(target_w2)[0].lemma_

    linked_sentences = []

    for sent in doc.sents:
        word_found1 = False
        word_found2 = False
        for token in sent:
            if token.lemma_ == target_lemma1:
                word_found1 = True
            if token.lemma_ == target_lemma2:
                word_found2 = True
        if word_found1 and word_found2:
            linked_sentences.append(sent.text)

    print(f"Contextos que incluyen '{target_w1}' y '{target_w2}' :\n")
    for sentence in linked_sentences:
        print(sentence)


In [ ]:
palabra = "persona" #@param {type:"string"}
checker(globals()[corpus_22], neologismo, palabra)